# Deep MTJ - automatic tracking of the muscel-tendon junction

This notebook shows how to use the MTJ tracking tool. Google Colab offers free online computation resources. You can track your own videos by uploading your video files (see menu on the left) and running the model online. It is also possible to run the tool on your local enviroment by installing the `setup.py` file (see GitHub repository; https://github.com/luuleitner/deepMTJ).

The first step when running in Colab is to go to "Runtime" and "change runtime type" under "Hardware accelerator" -> select "GPU" 

## Setup

In [ ]:
# deepMTJ can be directly installed from the GitHub repository using PIP.

!pip install git+https://github.com/luuleitner/deepMTJ 

In [ ]:
# We need to import necessary packages into Colab. Don't worry to much about this cell of code
# For tracking the MTJ positions from multiple videos we can simple use the `track_videos` function.

from mtj_tracking.predict.predict import track_videos # main tracking function
from mtj_tracking.data.loader import Frame, loadVideo, adjustFrame # helper for DeepMTJ
from mtj_tracking.process.filter import pre_filter, hampel_filter # load filter for time series data
from matplotlib import pyplot as plt # plotting function
import numpy as np # array handeling
from urllib.request import urlretrieve # for downloading the sample data
from enum import Enum # Enum package to add unknown framecuts

## Download sample data

In [ ]:
# Download a sample video from an Esaote instrument.

urlretrieve('https://storage.googleapis.com/deepmtj/IEEEtbme_testset_2021/deepMTJ_TS_v0001.avi', 'esaote_video.avi')

## Prepare your data

First, we need to specify the region of interest within the video. We select the x/y-position and the width and height of the frame that we want to crop from the video (x, y, width, height). The proportions of width/height need to have a ratio of 2/1.

We provide the following predefined crops:

```
Frame.ESAOTE = (185, 128, 580, 290)
Frame.TELEMED = (88, 120, 446, 223)
Frame.AIXPLORER = (200, 261, 1000, 500)
```

In [ ]:
# If you are not satisfied with any of the predefined crops just add your own:
# We have added the Ultrasonix ultrasound below.

class CustomFrame(Enum):
     USX_ST = (113, 185, 500, 200)  # Ultrasonix SonixTouch
     USX_RP = (126, 250, 490, 245)  # Ultrasonix RP

     #--> to define your own crop configure and uncomment the line below --->
     # YOUR_CROP_NAME = (x, y, width, height) # make sure that the proportions of width/height are 2/1

In [ ]:
frame = Frame.ESAOTE # apply the defined crop

#--> if you defined your own crop configure and uncomment the line below ---> 
# frame = CustomFrame.YOUR_CROP_NAME 

Let us check that we have selected the correct sub-frame by reading the video.

In [ ]:
video_id, video = loadVideo('esaote_video.avi') # returns the video id and a list of frames
adjusted_video_frame = adjustFrame(video[0], frame, (128, 256)) # crops to specified frame

plt.subplot(121)
plt.imshow(video[0]) # plot the first frame
plt.title('Full Frame')
plt.subplot(122)
plt.imshow(adjusted_video_frame, cmap='gray') # plot the adjusted frame
plt.title('Cropped Frame')
plt.show()

## Track the MTJ in single video

In [ ]:
# Now we can track the MTJ in our video.

result_df = track_videos(['esaote_video.avi'], frame)
result_df

In [ ]:
# Plot the tracked results of the video sequence

for i in range(len(video)):
    last_video_frame = adjustFrame(video[i], frame, (128, 256)) # crops to specified frame
    plt.imshow(last_video_frame, cmap='gray') # plot the adjusted frame
    plt.axis('off')
    plt.scatter(result_df.x[i], result_df.y[i], color='red')
    plt.show()

## Convert labels to full resolution and save Predictions

In [ ]:
# The result can also be converted to the pixel coordinates of the original video frame

result_oc_df = track_videos(['esaote_video.avi'], frame, frame_coordinates=False)
result_oc_df

In [ ]:
# Plot the first video frame in full resolution

plt.imshow(video[0]) # plot the original frame
plt.scatter(result_oc_df.x[0], result_oc_df.y[0], color='red')
plt.show()

In [ ]:
# Download the csv file using the Colab file explorer
result_df.to_csv('deep_mtj.csv')

## Track MTJ in multiple video files

To convert your own files, upload your videos and specify the list of video files in the `track_video` function.
The result can be afterwards saved as a CSV file and downloaded.

In [ ]:
result_df = track_videos(['<<your-video-1>>', '<<your-video-2>>'], frame)
result_df.to_csv('deep_mtj.csv') # Save the tracking results to a csv file

## Filter timeseries

These filter routines are just a standard template. You may need to switch the individual stages on and off or adjust the parameters depending on your data and recording quality.

In [ ]:
# If your data has NaN values (no prediction) this line might be helpfull to interpolate over missing sample points

result_df_interpolated = result_df[['x','y']].interpolate(method='linear', axis=0, limit = 20).ffill().bfill()

In [ ]:
# If your predictions are noisy we provide two types of filters to get rid of outliers

# Apply Prefilter
result_df_prefiltered = pre_filter(result_df_interpolated)

# Apply Hampel Filter
result_df_filtered = np.column_stack(([hampel_filter(result_df_prefiltered.iloc[:, col], 10, n_sigmas=2) for col in range(result_df_prefiltered.shape[1])]))

In [ ]:
# Plot your filter results here

fig = plt.figure(figsize=(6, 7), dpi=300)
ax_1 = fig.add_subplot(211)

# MTJ x-coordinate
ax_1.plot(result_df['x'], c='g', label='raw') # raw prediction
ax_1.plot(result_df_interpolated['x'], c='b', label='NaN interp.') # interpolated prediction
ax_1.plot(result_df_filtered[:, 0], c='r', label='filtered') # filterd prediction
ax_1.xaxis.set_visible(False)
ax_1.set_ylabel('MTU x-position')
ax_1.legend(loc='lower left')
ax_1.set_title('MTJ x-coordinate')
# MTJ y-coordinate
ax_2 = fig.add_subplot(212)
ax_2.plot(result_df['y'], c='g', label='raw') # raw prediction
ax_2.plot(result_df_interpolated['y'], c='b', label='NaN interp.') # interpolated prediction
ax_2.plot(result_df_filtered[:, 1], c='r', label='filtered') # filterd prediction
ax_2.xaxis.set_visible(False)
ax_2.set_ylabel('MTJ y-position')
ax_2.legend(loc='lower left')
ax_2.set_title('MTJ y-coordinate')
#
plt.tight_layout()
plt.show()